Rain Forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Load the dataset
dataset = pd.read_csv("C:\Users\dalow\Downloads\processed_file.csv")



y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))